In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd
import numpy as np

In [65]:
x1 = [random.gauss(0, 1) for _ in range(10000)] 
x2 = [random.gauss(0, 1) for _ in range(10000)]
y = [0.25*x1[i]+0.25*x2[i]+random.gauss(0, 1)  for i in range(len(x1))]

data = pd.DataFrame({'x1':x1, 'x2':x2, 'y': y})

In [67]:
data.mean()

x1    0.002836
x2   -0.000030
y    -0.002423
dtype: float64

In [66]:
data.std()

x1    1.010403
x2    1.000040
y     1.065001
dtype: float64

In [68]:
data.corr()

,x1,x2,y
x1,1.000000,0.017221,0.234664
x2,0.017221,1.000000,0.244501
y,0.234664,0.244501,1.000000


In [69]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = torch.tensor(self.x.iloc[idx].values, dtype=torch.float32)
        y = torch.tensor(self.y.iloc[idx], dtype=torch.float32)
        return x, y

In [ ]:
class DenseNeuralNetwork(nn.Module):
    def __init__(self, 
            n_hidden_layers, 
            hidden_layer_n_neurons, 
            activation_function, 
            dropout_prob, 
            input_size, 
            output_size, 
            batch_normalisation,
            last_activation_function = 'relu',):

        super(DenseNeuralNetwork, self).__init__()

        self.n_hidden_layers = n_hidden_layers
        self.batch_normalisation = batch_normalisation

        self.ACTIVATION_FUNCTIONS_MAP = {'relu': nn.ReLU(), 
                                        'sigmoid': nn.Sigmoid(), 
                                        'tanh': nn.Tanh(), 
                                        'softmax': nn.Softmax(dim=1)}

        # Define layers
        self.nn_layers = nn.ModuleList([nn.Linear(input_size, hidden_layer_n_neurons[0])])
        if self.batch_normalisation:  
            self.batch_norm_layers = nn.ModuleList([])
        self.activation_layer = nn.ModuleList([self.ACTIVATION_FUNCTIONS_MAP[activation_function]])
        self.dropout_layer = nn.ModuleList([nn.Dropout(dropout_prob)])
        
        for i in range(n_hidden_layers-1):
            self.nn_layers.append(nn.Linear(hidden_layer_n_neurons[i], hidden_layer_n_neurons[i+1]))
            self.batch_norm_layers.append(nn.BatchNorm1d(hidden_layer_n_neurons[i + 1]))
            self.activation_layer.append(self.ACTIVATION_FUNCTIONS_MAP[activation_function])
            self.dropout_layer.append(nn.Dropout(dropout_prob))

        self.nn_layers.append(nn.Linear(hidden_layer_n_neurons[-1], output_size))
        self.activation_layer.append(self.ACTIVATION_FUNCTIONS_MAP[last_activation_function])


    def forward(self, x, training=True):
        out = self.nn_layers[0](x)

        for i in range(self.n_hidden_layers):
            if i != 0:
                out = self.nn_layers[i](out)
            
            if self.batch_normalisation:
                if i != 0:  # Exclude batch normalization for the input layer
                    out = self.batch_norm_layers[i - 1](out)

            out = self.activation_layer[i](out)

            if training:
                out = self.dropout_layer[i](out)
            
        out = self.nn_layers[-1](out)
        out = self.activation_layer[-1](out)
        
        return out

In [70]:
class DNN_const_pt:
    
    def __init__(self, 
                 n_hidden_layers, 
                 activation, 
                 lambda_lasso, 
                 epoch_batch_size, 
                 batch_size,
                 learning_rate, 
                 num_epochs, 
                 random_state, 
                 dropout_prob,
                 hidden_layer_n_neuron,
                 batch_normalisation = False,
                 verbose = False,
                 gpu = False,
                 gpu_id = 0,
                 loss_function='MSE',
                 last_activation_function = 'relu',
                 **kwargs):

        self.n_hidden_layers = n_hidden_layers
        self.activation = activation
        self.lambda_lasso = lambda_lasso
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.epoch_batch_size = epoch_batch_size
        self.batch_size = batch_size
        self.random_state = random_state
        self.hidden_layer_n_neuron = hidden_layer_n_neuron
        self.dropout_prob = dropout_prob
        self.verbose = verbose
        self.loss_function = loss_function
        self.last_activation_function = last_activation_function
        self.batch_normalisation = batch_normalisation
        
        self.gpu = gpu
        self.gpu_id = gpu_id


        self.LOSS_FUNCTIONS_MAP = {'MSE': nn.MSELoss(), 
                                    'MAE': nn.L1Loss(), 
                                    'Huber': nn.SmoothL1Loss()}
    


    def fit(self, train_x, train_y, initial_model = None):


        self.hidden_layer_sizes = [self.hidden_layer_n_neuron for i in range(self.n_hidden_layers)]

        if type(train_y) == pd.core.frame.DataFrame:
            self.output_size = len(train_y.columns)
        else:
            self.output_size = 1
        
        self.input_size = len(train_x.columns)

       
        # Create the model
        self.model = DenseNeuralNetwork(self.n_hidden_layers, 
                                        self.hidden_layer_sizes, 
                                        self.activation, 
                                        self.dropout_prob, 
                                        self.input_size, 
                                        self.output_size,
                                        self.last_activation_function,
                                        self.batch_normalisation)

        if initial_model is not None:
            self.model.load_state_dict(initial_model.model.state_dict())        
        
        if self.gpu:
            self.model.cuda(self.gpu_id)


        # Define the loss function and optimizer
        criterion = self.LOSS_FUNCTIONS_MAP['MSE']
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

        # Create the custom datasets
        train_dataset = CustomDataset(train_x, train_y)

        self.seeds = np.random.randint(10000000, size = self.num_epochs)

        

        # Training loop
        for epoch in range(self.num_epochs):
            
            train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, worker_init_fn=lambda _: torch.manual_seed(self.seeds[epoch]))

            instances_learnt_so_far = 0

            for batch_idx, (batch_train_x, batch_train_y) in enumerate(train_loader):
                
                if self.gpu:
                    batch_train_x, batch_train_y = batch_train_x.cuda(self.gpu_id), batch_train_y.cuda(self.gpu_id)
                # Forward pass
                outputs = self.model(batch_train_x)
                target = batch_train_y.view(-1, 1)  # Reshape target tensor to match the size of the output
                loss = criterion(outputs, target)

                # Lasso regularization
                l1_regularization = torch.tensor(0.)
                for param in self.model.parameters():
                    l1_regularization += torch.norm(param, p=1)
                loss += self.lambda_lasso * l1_regularization

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                instances_learnt_so_far += self.batch_size
                
                if instances_learnt_so_far >= self.epoch_batch_size:
                    break
        
            # Print the progress
            if self.verbose:
                if (epoch + 1) % 100 == 0:
                    print(f'Epoch [{epoch+1}/{self.num_epochs}], Loss: {loss.item():.4f}')

    def predict(self, x):

        x_tensor = torch.Tensor(x.values)

        if self.gpu:
            x_tensor = x_tensor.cuda(self.gpu_id)

        with torch.no_grad():
            predictions = self.model(x_tensor, training=False)

        return predictions
    




class DNN_shrink_pt:
    
    def __init__(self, 
                 n_hidden_layers, 
                 activation, 
                 lambda_lasso, 
                 epoch_batch_size, 
                 batch_size,
                 learning_rate, 
                 num_epochs, 
                 random_state, 
                 dropout_prob,
                 batch_normalisation = False,
                 verbose = False,
                 gpu = False,
                 gpu_id = 0,
                 loss_function='MSE',
                 last_activation_function = 'relu',
                 **kwargs):

        self.n_hidden_layers = n_hidden_layers
        self.activation = activation
        self.lambda_lasso = lambda_lasso
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.epoch_batch_size = epoch_batch_size
        self.batch_size = batch_size
        self.random_state = random_state
        self.dropout_prob = dropout_prob
        self.verbose = verbose
        self.loss_function = loss_function
        self.last_activation_function = last_activation_function
        self.batch_normalisation = batch_normalisation
        
        self.gpu = gpu
        self.gpu_id = gpu_id


        self.LOSS_FUNCTIONS_MAP = {'MSE': nn.MSELoss(), 
                                    'MAE': nn.L1Loss(), 
                                    'Huber': nn.SmoothL1Loss()}
    


    def fit(self, train_x, train_y, initial_model=None):

        if type(train_y) == pd.core.frame.DataFrame:
            self.output_size = len(train_y.columns)
        else:
            self.output_size = 1
        
        self.input_size = len(train_x.columns)

        gap = (self.input_size - self.output_size)//(self.n_hidden_layers+1)
        self.hidden_layer_sizes = [self.input_size - i * gap for i in range(self.n_hidden_layers)]
    
        # Create the model
        self.model = DenseNeuralNetwork(self.n_hidden_layers, 
                                        self.hidden_layer_sizes, 
                                        self.activation, 
                                        self.dropout_prob, 
                                        self.input_size, 
                                        self.output_size,
                                        self.last_activation_function,
                                        self.batch_normalisation)
        
        if initial_model is not None:
            self.model.load_state_dict(initial_model.model.state_dict())
        
        if self.gpu:
            self.model.cuda(self.gpu_id)


        # Define the loss function and optimizer
        criterion = self.LOSS_FUNCTIONS_MAP['MSE']
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

        # Create the custom datasets
        train_dataset = CustomDataset(train_x, train_y)

        self.seeds = np.random.randint(10000000, size = self.num_epochs)

        

        # Training loop
        for epoch in range(self.num_epochs):
            
            train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, worker_init_fn=lambda _: torch.manual_seed(self.seeds[epoch]))

            instances_learnt_so_far = 0

            for batch_idx, (batch_train_x, batch_train_y) in enumerate(train_loader):
                
                if self.gpu:
                    batch_train_x, batch_train_y = batch_train_x.cuda(self.gpu_id), batch_train_y.cuda(self.gpu_id)
                # Forward pass
                outputs = self.model(batch_train_x)
                target = batch_train_y.view(-1, 1)  # Reshape target tensor to match the size of the output
                loss = criterion(outputs, target)

                # Lasso regularization
                l1_regularization = torch.tensor(0.)
                for param in self.model.parameters():
                    l1_regularization += torch.norm(param, p=1)
                loss += self.lambda_lasso * l1_regularization

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                instances_learnt_so_far += self.batch_size
                
                if instances_learnt_so_far >= self.epoch_batch_size:
                    break
        
            # Print the progress
            if self.verbose:
                if (epoch + 1) % 100 == 0:
                    print(f'Epoch [{epoch+1}/{self.num_epochs}], Loss: {loss.item():.4f}')

    def predict(self, x):

        x_tensor = torch.Tensor(x.values)

        if self.gpu:
            x_tensor = x_tensor.cuda(self.gpu_id)

        with torch.no_grad():
            predictions = self.model(x_tensor, training=False)

        return predictions

In [78]:
model = DNN_const_pt(n_hidden_layers = 1, 
                 activation = 'relu', 
                 lambda_lasso = 0, 
                 epoch_batch_size = 1000, 
                 batch_size = 1000,
                 learning_rate = 0.1, 
                 num_epochs = 200, 
                 random_state = 19260817, 
                 dropout_prob = 0,
                 hidden_layer_n_neuron = 2,
                 batch_normalisation = True,
                 verbose = True,
                 loss_function ='MSE',
                 last_activation_function = 'relu')

In [79]:
model.fit(data[['x1', 'x2']], data['y'])

Epoch [100/200], Loss: 1.1194
Epoch [200/200], Loss: 1.1100


In [80]:
list(model.model.parameters())

[Parameter containing:
 tensor([[-1.0253, -1.0685],
         [ 0.6198,  0.7410]], requires_grad=True),
 Parameter containing:
 tensor([-1.3482,  0.0124], requires_grad=True),
 Parameter containing:
 tensor([[-0.4405,  0.3528]], requires_grad=True),
 Parameter containing:
 tensor([-0.0665], requires_grad=True)]

In [81]:
from sklearn.metrics import r2_score
predict = model.predict(data[['x1', 'x2']])

predictions = [float(x[0]) for x in predict]
r2_score(data['y'], predictions)

0.0558707918244834

In [83]:
data['y']

0      -0.384704
1      -0.416314
2      -0.056596
3       0.915829
4       1.055674
          ...   
9995   -0.540624
9996    0.345753
9997   -1.387876
9998   -2.197660
9999   -1.948410
Name: y, Length: 10000, dtype: float64

In [82]:
predictions

[0.14526790380477905,
 0.0,
 0.0,
 0.14678797125816345,
 0.0,
 0.0,
 0.19516590237617493,
 0.0,
 0.0,
 0.0,
 0.0860733836889267,
 0.0,
 0.0,
 0.0,
 0.2246960997581482,
 0.6782543659210205,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.1255495250225067,
 0.0,
 0.0,
 0.0,
 0.0,
 0.33527037501335144,
 0.0,
 0.0,
 0.010418549180030823,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5756535530090332,
 0.15444590151309967,
 0.0,
 0.16982242465019226,
 0.0,
 0.0,
 0.003875739872455597,
 0.0,
 0.0,
 0.0,
 0.3394787311553955,
 0.0,
 0.0,
 0.0,
 0.0,
 0.14277370274066925,
 0.15242500603199005,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.16750560700893402,
 0.5207821726799011,
 0.0,
 0.11467058956623077,
 0.07092072069644928,
 0.0,
 0.0,
 0.3659733533859253,
 0.04576386511325836,
 0.0,
 0.14024102687835693,
 0.27303028106689453,
 0.0,
 0.0,
 0.0,
 0.5224831700325012,
 0.040009960532188416,
 0.1842065155506134,
 0.0,
 0.0,
 0.0,
 0.6662347912788391,
 0.2561276853084564,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.14367248117923

In [76]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(data[['x1', 'x2']], data['y'])

In [77]:
r2_score(data['y'], model.predict(data[['x1', 'x2']]))

0.11290521370902162